# Dataset

Utilizando las librerias BeautifulSoup 4, request y csv

In [6]:
from bs4 import BeautifulSoup
import requests
from csv import writer
from time import time, sleep


Data extracted from: [MercadoLibre Argentina](https://inmuebles.mercadolibre.com.ar/casas/venta/#CATEGORY_ID=MLA1468&S=hc_inmuebles&c_id=undefined&c_element_order=undefined&c_campaign=VENTA&c_uid=590c8450-6823-11ed-8220-c18bb10563ed) 19/11/2022


Zonas tenidas en cuenta:
- Bs.As GBA Norte
- Bs.As. GBA Sur
- Bs.As. GBA Oeste


Variables tenidas en cuenta:
- Zona (Zone)
- Localidad (Locality)
- Metros al cuadrado (Meters squared)
- Cant. de ambientes (Number of rooms)
- Valor en dolares (Price in dollars)

In [7]:
def busqueda_plus(zona, localidades, pags):
    for localidad in localidades:
        for i in range(1,(48*pags)+1,48):
            sleep(3)
            url = f"https://inmuebles.mercadolibre.com.ar/casas/venta/bsas_gba-{zona}/{localidad}/_Desde_{i}_NoIndex_True"
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            houses = soup.find_all("div", "ui-search-result__content-wrapper shops__result-content-wrapper")
            for x in houses:
                loc = x.find("span", class_="ui-search-item__group__element ui-search-item__location shops__items-group-details").text.split(",")[-2].replace(" ", "")
                info = x.find("ul", class_="ui-search-card-attributes ui-search-item__group__element shops__items-group-details").text.split()
                mt2 = info[0].replace(",", "")
                amb = info[-2][-1]
                if amb == "0":
                    amb = "10"
                elif amb == "²":
                    amb = "0"
                elif amb == "a":
                    amb = "0"
                prec = x.find("span", class_="price-tag-fraction").text.replace(".", "")
                row = [zona, loc, mt2, amb, prec]
                with open("gba_houses__.csv", "a", encoding="utf8", newline="") as f:
                    w = writer(f)
                    w.writerow(row)


Cración del dataset con formato .csv

In [8]:
with open("gba_houses__.csv", "w", encoding="utf8", newline="") as f:
    w = writer(f)
    header = ["zona", "loc", "mtr2", "ambs", "valor"]
    w.writerow(header)
    

Relleno del dataset con una seleccion de localidades de cada zona.

In [9]:
norte = ["pilar", "tigre", "escobar", "san-isidro", "san-miguel", "vicente-lopez", "general-san-martin", "malvinas-argentinas", "san-fernando", "jose-c-paz"]
sur = ["la-plata", "esteban-echeverria", "quilmes", "lanus", "ezeiza", "lomas-de-zamora", "berazategui", "almirante-brown", "avellaneda"]
oeste = ["la-matanza", "ituzaingo", "moreno", "moron", "castelar", "merlo", "hurlingham", "tres-de-febrero"]

print("Creating Dataset...")
start = time()
busqueda_plus("Norte", norte, 22)
busqueda_plus("Sur", sur, 26)
busqueda_plus("Oeste", oeste, 28)
end = time()
timer = end - start
print("Dataset Created!")
print(f"Tiempo tardado: {timer/60} minutos.")


Creating Dataset...
Dataset Created!
Tiempo tardado: 63.19745614528656 minutos.
